In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-37449?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Retail Associate, Seas

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsRetail StoresRetail Associate, SeasDetroit, MichiganBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does 

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})


# json_parser = JsonOutputParser()
# json_res = json_parser.parse(res.content)

# for i in json_res:
#     new_json_file=i

# return new_json_file

str

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Retail Associate, Seas',
  'experience': 'No experience required',
  'skills': ['Customer service',
   'Product knowledge',
   'Teamwork',
   'Communication',
   'Problem-solving',
   'Time management',
   'Adaptability',
   'Physical stamina'],
  'description': 'As a Nike Retail Associate, you’re the face of NIKE. Enjoy high-volume and a fast pace as your diverse experience and perspective helps guide customers in making the best decisions for them. You’ll work with your team to focus on customer service and get to the win the right way.'}]

In [7]:
type(json_res)

list

In [9]:
for i in json_res:
    new_json_file=i

In [10]:
new_json_file

{'role': 'Retail Associate, Seas',
 'experience': 'No experience required',
 'skills': ['Customer service',
  'Product knowledge',
  'Teamwork',
  'Communication',
  'Problem-solving',
  'Time management',
  'Adaptability',
  'Physical stamina'],
 'description': 'As a Nike Retail Associate, you’re the face of NIKE. Enjoy high-volume and a fast pace as your diverse experience and perspective helps guide customers in making the best decisions for them. You’ll work with your team to focus on customer service and get to the win the right way.'}

In [11]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df


,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [12]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [14]:
links = collection.query(query_texts=new_json_file['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}]]

In [15]:
new_json_file

{'role': 'Retail Associate, Seas',
 'experience': 'No experience required',
 'skills': ['Customer service',
  'Product knowledge',
  'Teamwork',
  'Communication',
  'Problem-solving',
  'Time management',
  'Adaptability',
  'Physical stamina'],
 'description': 'As a Nike Retail Associate, you’re the face of NIKE. Enjoy high-volume and a fast pace as your diverse experience and perspective helps guide customers in making the best decisions for them. You’ll work with your team to focus on customer service and get to the win the right way.'}

In [17]:
job = new_json_file
job['skills']

['Customer service',
 'Product knowledge',
 'Teamwork',
 'Communication',
 'Problem-solving',
 'Time management',
 'Adaptability',
 'Physical stamina']

In [18]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Apil, a business development executive at Apil Bussiness Development company. Apil Bussiness Development company is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Apil Bussiness Development company 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Apil Bussiness Development company's portfolio: {link_list}
        Remember you are Apil, BDE at Apil Bussiness Development company. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhance Your Retail Operations with Apil Business Development Company

Dear Hiring Manager,

I came across the job description for a Retail Associate at Nike and was impressed by the emphasis on customer service, teamwork, and problem-solving skills. As a Business Development Executive at Apil Business Development Company, I believe our expertise in AI and software consulting can help Nike streamline its retail operations and enhance the overall customer experience.

Our company has a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. We can help Nike leverage technology to improve inventory management, customer engagement, and employee productivity.

Some of our notable projects include:

* Magento e-commerce solutions that enable seamless online shopping experiences (https://example.com/magento-portfolio)
* Machine learning and Python-based applications that d